# File Inspection (Experimental)

**by Josh Dillon & Aaron Parsons**, last updated August 8, 2022

In [1]:
import numpy as np
import os
from hera_qm import ant_metrics, ant_class
from hera_cal import io, utils
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Parse inputs

To use this notebook interactively, you will have to provide a sum filename path if none exists as an environment variable. All other parameters have reasonable default values.


In [2]:
# get file names
SUM_FILE = os.environ.get("SUM_FILE", None)
# SUM_FILE = '/mnt/sn1/zen.2459797.30001.sum.uvh5'  # If sum_file is not defined in the environment variables, define it here.
DIFF_FILE = SUM_FILE.replace('sum', 'diff')
print(f"SUM_FILE = '{SUM_FILE}'")

SUM_FILE = '/mnt/sn1/2459802/zen.2459802.25321.sum.uvh5'


### Parse Bounds

In [3]:
# ant_metrics bounds for low correlation / dead antennas
AM_CORR_BAD = float(os.environ.get("AM_CORR_BAD", 0.2))
AM_CORR_SUSPECT = float(os.environ.get("AM_CORR_SUSPECT", 0.4))
am_corr_bad = (0, AM_CORR_BAD)
am_corr_suspect = (AM_CORR_BAD, AM_CORR_SUSPECT)

# ant_metrics bounds for cross-polarized antennas
AM_XPOL_BAD = float(os.environ.get("AM_XPOL_BAD", -0.1))
AM_XPOL_SUSPECT = float(os.environ.get("AM_XPOL_SUSPECT", 0))
am_xpol_bad = (-1, AM_XPOL_BAD)
am_xpol_suspect = (AM_XPOL_BAD, AM_XPOL_SUSPECT)

# bounds on autocorrelation power
AUTO_POWER_GOOD_LOW = float(os.environ.get("AUTO_POWER_GOOD_LOW", 5))
AUTO_POWER_GOOD_HIGH = float(os.environ.get("AUTO_POWER_GOOD_HIGH", 30))
auto_power_good = (AUTO_POWER_GOOD_LOW, AUTO_POWER_GOOD_HIGH)
AUTO_POWER_SUSPECT_LOW = float(os.environ.get("AUTO_POWER_SUSPECT_LOW", 1))
AUTO_POWER_SUSPECT_HIGH = float(os.environ.get("AUTO_POWER_SUSPECT_HIGH", 80))
auto_power_suspect = (AUTO_POWER_SUSPECT_LOW, AUTO_POWER_SUSPECT_HIGH)

# bounds on autocorrelation slope
AUTO_SLOPE_GOOD_LOW = float(os.environ.get("AUTO_SLOPE_GOOD_LOW", -0.2))
AUTO_SLOPE_GOOD_HIGH = float(os.environ.get("AUTO_SLOPE_GOOD_HIGH", 0.2))
auto_slope_good = (AUTO_SLOPE_GOOD_LOW, AUTO_SLOPE_GOOD_HIGH)
AUTO_SLOPE_SUSPECT_LOW = float(os.environ.get("AUTO_SLOPE_SUSPECT_LOW", -0.4))
AUTO_SLOPE_SUSPECT_HIGH = float(os.environ.get("AUTO_SLOPE_SUSPECT_HIGH", 0.4))
auto_slope_suspect = (AUTO_SLOPE_SUSPECT_LOW, AUTO_SLOPE_SUSPECT_HIGH)

for bound in ['am_corr_bad', 'am_corr_suspect', 'am_xpol_bad', 'am_xpol_suspect', 
             'auto_power_good', 'auto_power_suspect', 'auto_slope_good', 'auto_slope_suspect']:
    print(f'{bound} = {eval(bound)}')

am_corr_bad = (0, 0.2)
am_corr_suspect = (0.2, 0.4)
am_xpol_bad = (-1, -0.1)
am_xpol_suspect = (-0.1, 0.0)
auto_power_good = (5.0, 30.0)
auto_power_suspect = (1.0, 80.0)
auto_slope_good = (-0.2, 0.2)
auto_slope_suspect = (-0.4, 0.4)


## Load sum and diff data

In [4]:
hd = io.HERADataFastReader(SUM_FILE)
data, _, _ = hd.read(read_flags=False, read_nsamples=False)
hd_diff = io.HERADataFastReader(DIFF_FILE)
diff_data, _, _ = hd_diff.read(read_flags=False, read_nsamples=False)

In [5]:
ants = sorted(set([ant for bl in hd.bls for ant in utils.split_bl(bl)]))
auto_bls = [bl for bl in data if (bl[0] == bl[1]) and (utils.split_pol(bl[2])[0] == utils.split_pol(bl[2])[1])]

In [6]:
print(f'File: {SUM_FILE}')
print(f'JDs: {hd.times} ({np.median(np.diff(hd.times)) * 24 * 3600:.5f} s integrations)')
print(f'LSTS: {hd.lsts * 12 / np.pi } hours')
print(f'Frequencies: {len(hd.freqs)} {np.median(np.diff(hd.freqs)) / 1e6:.5f} MHz channels from {hd.freqs[0] / 1e6:.5f} to {hd.freqs[-1] / 1e6:.5f} MHz')
print(f'Antennas: {len(hd.data_ants)}')
print(f'Polarizations: {hd.pols}')

File: /mnt/sn1/2459802/zen.2459802.25321.sum.uvh5
JDs: [2459802.25315493 2459802.25326678] (9.66368 s integrations)
LSTS: [16.78410484 16.78679654] hours
Frequencies: 1536 0.12207 MHz channels from 46.92078 to 234.29871 MHz
Antennas: 147
Polarizations: ['nn', 'ee', 'ne', 'en']


## Run `ant_metrics`

In [7]:
am = ant_metrics.AntennaMetrics(SUM_FILE, DIFF_FILE, sum_data=data, diff_data=diff_data)
am.iterative_antenna_metrics_and_flagging(crossCut=am_xpol_bad[1], deadCut=am_corr_bad[1])

/home/obs/anaconda/envs/RTP/lib/python3.9/site-packages/hera_qm/ant_metrics.py:129: RuntimeWarning: invalid value encountered in divide
  odd /= np.abs(odd)
/home/obs/anaconda/envs/RTP/lib/python3.9/site-packages/hera_qm/ant_metrics.py:128: RuntimeWarning: invalid value encountered in divide
  even /= np.abs(even)
/home/obs/anaconda/envs/RTP/lib/python3.9/site-packages/hera_qm/ant_metrics.py:128: RuntimeWarning: invalid value encountered in divide
  even /= np.abs(even)
/home/obs/anaconda/envs/RTP/lib/python3.9/site-packages/hera_qm/ant_metrics.py:129: RuntimeWarning: invalid value encountered in divide
  odd /= np.abs(odd)
/home/obs/anaconda/envs/RTP/lib/python3.9/site-packages/hera_qm/ant_metrics.py:129: RuntimeWarning: invalid value encountered in divide
  odd /= np.abs(odd)
/home/obs/anaconda/envs/RTP/lib/python3.9/site-packages/hera_qm/ant_metrics.py:129: RuntimeWarning: invalid value encountered in divide
  odd /= np.abs(odd)
/home/obs/anaconda/envs/RTP/lib/python3.9/site-package

In [8]:
totally_dead_ants = [ant for ant, i in am.removal_iteration.items() if i == -1]
am_totally_dead = ant_class.AntennaClassification(good=[ant for ant in ants if ant not in totally_dead_ants], bad=totally_dead_ants)
am_corr = ant_class.antenna_bounds_checker(am.final_metrics['corr'], bad=[am_corr_bad], suspect=[am_corr_suspect], good=[(0, 1)])
am_xpol = ant_class.antenna_bounds_checker(am.final_metrics['corrXPol'], bad=[am_xpol_bad], suspect=[am_xpol_suspect], good=[(-1, 1)])
ant_metrics_class = am_totally_dead + am_corr + am_xpol
print('ant_metrics Classification:\n\n', ant_metrics_class)

ant_metrics Classification:

 Jee:
----------
good (78 antpols):
3, 4, 5, 7, 8, 9, 10, 15, 16, 17, 18, 19, 20, 21, 29, 30, 31, 32, 33, 36, 40, 41, 42, 45, 46, 50, 51, 52, 53, 54, 55, 56, 69, 70, 71, 72, 81, 82, 83, 84, 85, 86, 87, 98, 99, 100, 101, 103, 104, 109, 111, 116, 117, 118, 119, 121, 122, 123, 127, 128, 129, 141, 156, 157, 158, 161, 162, 168, 169, 170, 176, 177, 178, 179, 180, 182, 189, 191

suspect (16 antpols):
28, 92, 94, 102, 110, 112, 120, 130, 142, 167, 190, 321, 323, 324, 329, 333

bad (53 antpols):
27, 37, 38, 57, 65, 66, 67, 68, 73, 88, 90, 91, 93, 105, 106, 107, 108, 125, 126, 135, 136, 137, 138, 140, 143, 144, 145, 150, 155, 160, 163, 164, 165, 166, 181, 183, 184, 185, 186, 187, 203, 205, 206, 207, 220, 221, 222, 223, 224, 241, 242, 243, 320


Jnn:
----------
good (74 antpols):
3, 4, 5, 7, 8, 9, 10, 15, 16, 17, 19, 20, 21, 29, 30, 31, 32, 36, 40, 41, 42, 45, 46, 50, 51, 52, 53, 54, 55, 56, 69, 70, 71, 72, 73, 81, 82, 83, 84, 85, 86, 87, 98, 99, 100, 101, 102, 103, 1

## Examine and classify autocorrelations

In [9]:
auto_power_class = ant_class.auto_power_checker(data, good=auto_power_good, suspect=auto_power_suspect)
auto_slope_class = ant_class.auto_slope_checker(data, good=auto_slope_good, suspect=auto_slope_suspect, edge_cut=100, filt_size=17)
auto_class = auto_power_class + auto_slope_class
print('Autocorrelation Classification:\n\n', auto_class)

Autocorrelation Classification:

 Jee:
----------
good (79 antpols):
3, 4, 5, 7, 9, 10, 15, 16, 17, 18, 19, 20, 21, 28, 29, 30, 31, 33, 40, 41, 42, 45, 46, 51, 53, 54, 55, 56, 69, 71, 72, 81, 82, 83, 85, 86, 93, 94, 98, 100, 103, 109, 111, 112, 116, 117, 118, 120, 121, 127, 128, 129, 130, 141, 143, 144, 156, 157, 158, 161, 162, 163, 164, 165, 166, 176, 177, 178, 179, 180, 184, 185, 186, 187, 189, 191, 206, 329, 333

suspect (23 antpols):
8, 36, 52, 70, 84, 99, 101, 102, 104, 119, 122, 123, 167, 168, 169, 170, 182, 205, 207, 223, 321, 323, 324

bad (45 antpols):
27, 32, 37, 38, 50, 57, 65, 66, 67, 68, 73, 87, 88, 90, 91, 92, 105, 106, 107, 108, 110, 125, 126, 135, 136, 137, 138, 140, 142, 145, 150, 155, 160, 181, 183, 190, 203, 220, 221, 222, 224, 241, 242, 243, 320


Jnn:
----------
good (79 antpols):
3, 4, 5, 7, 9, 10, 15, 16, 17, 18, 19, 20, 21, 29, 30, 31, 33, 40, 41, 42, 45, 46, 50, 51, 54, 55, 56, 69, 70, 71, 72, 73, 81, 82, 83, 85, 86, 93, 94, 98, 99, 100, 102, 109, 110, 111, 112

## Metadata

In [10]:
from hera_cal import __version__
print('hera_cal:', __version__)
from hera_qm import __version__
print('hera_qm:', __version__)

hera_cal: 3.1.4.dev3+g68bd8c3
hera_qm: 2.0.3.dev44+g7d4aa18
